In [3]:
# ============================================================
# SIGNAL-BASED VOLLEYBALL ANALYSIS
# Ball Time-Series + Player Foot Circles
# ============================================================

import cv2
import numpy as np
import torch
from ultralytics import YOLO
from collections import deque

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
yolo = YOLO("yolov8n.pt").to(DEVICE)

PERSON_ID = 0
BALL_ID = 32

# -------- EDIT THESE --------
COURT_POLY = np.array([
    [313,292],
    [996,289],
    [1198,708],
    [125,707]
], dtype=np.int32)

SPLIT_Y = 400
# ----------------------------

VIDEO_PATH = "/home/naman/Cryptonite-RTP-NamanGoel/Task-0/volleyball_match.mp4"
OUTPUT_PATH = "/home/naman/Cryptonite-RTP-NamanGoel/Task-0/volleyball_match_output_mean_approach.mp4"

cap = cv2.VideoCapture(VIDEO_PATH)
FPS = int(cap.get(cv2.CAP_PROP_FPS))
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
FRAME_AREA = W*H

out = cv2.VideoWriter(
    OUTPUT_PATH,
    cv2.VideoWriter_fourcc(*"mp4v"),
    FPS,
    (W,H)
)

# Rolling buffers
ball_series = deque(maxlen=15)
player_series = {}

def inside_court(cx,cy):
    return cv2.pointPolygonTest(COURT_POLY,(float(cx),float(cy)),False)>=0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo(frame, device=DEVICE, verbose=False)

    current_players = []

    ball_point = None
    best_area = 0

    for r in results:
        for b in r.boxes:
            cls = int(b.cls[0])
            conf = float(b.conf[0])
            x1,y1,x2,y2 = b.xyxy[0].cpu().numpy().astype(int)
            cx,cy = (x1+x2)//2,(y1+y2)//2
            area = (x2-x1)*(y2-y1)

            # PERSON
            if cls==PERSON_ID and conf>0.45:
                if not inside_court(cx,cy):
                    continue
                if area<2000:
                    continue
                height = y2-y1
                foot = (cx,y2)
                current_players.append((foot,height))

            # BALL
            if cls==BALL_ID and conf>0.12:
                if 50<area<0.01*FRAME_AREA and area>best_area:
                    best_area=area
                    ball_point=(cx,cy)

    # -------- BALL TIME SERIES --------
    if ball_point:
        ball_series.append(ball_point)

    if len(ball_series)>0:
        avg_x = int(np.mean([p[0] for p in ball_series]))
        avg_y = int(np.mean([p[1] for p in ball_series]))
        cv2.circle(frame,(avg_x,avg_y),6,(0,255,255),-1)

    # -------- PLAYER PROCESSING --------
    teamA=0
    teamB=0

    for foot,height in current_players:
        cx,cy = foot
        if cy>SPLIT_Y:
            team="A"
            color=(0,255,0)
            teamA+=1
        else:
            team="B"
            color=(255,0,0)
            teamB+=1

        cv2.circle(frame,(cx,cy),10,color,-1)

    # -------- OVERLAY --------
    cv2.polylines(frame,[COURT_POLY],True,(255,255,255),2)
    cv2.line(frame,(0,SPLIT_Y),(W,SPLIT_Y),(0,0,255),2)

    cv2.putText(frame,f"Team A: {teamA}",(20,30),
                cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),2)
    cv2.putText(frame,f"Team B: {teamB}",(20,60),
                cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)

    out.write(frame)

cap.release()
out.release()
print("DONE: Signal-based output saved.")


DONE: Signal-based output saved.
